In [ ]:
import io, sys, os, datetime, requests
from collections import defaultdict
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import shapely
import boto3
import geopandas as gpd
import io

In [ ]:
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import geemap
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
OUTPUT_FILENAME = 'AQ-2-highpollutiondays.csv'

In [ ]:
SPECIES_INFO = {
    'no2': {
        'name': 'nitrogen dioxide',
        'molar_mass': 46.0055,
        'cams_unit': 'kg/kg',
        'who_threshold': 25.0,
    },
    'so2': {
        'name': 'sulfur dioxide',
        'molar_mass': 64.066,
        'cams_unit': 'kg/kg',
        'who_threshold': 40.0
    },
    'o3': {    # Ozone thresholds are based on 8-hour average, not 24-hour.
               # We use averages at 9am, noon, 3pm to get a 9-hour average at peak O3 production.
        'name': 'ozone',
        'molar_mass': 48.0,
        'cams_unit': 'kg/kg',
        'who_threshold': 100.0
    },
    'pm25': {
        'name': 'fine particulate matter',
        'cams_unit': 'kg/m^3',
        'who_threshold': 15.0
    },
    'pm10': {
        'name': 'coarse particulate matter',
        'cams_unit': 'kg/m^3',
        'who_threshold': 45.0
    },
    'co': {
        'name': 'carbon monoxide',
        'molar_mass': 28.01,
        'cams_unit': 'kg/kg',
        'who_threshold': 4000.0
    }
}

datasets = defaultdict(None)

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.client(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"

In [ ]:
for species in SPECIES_INFO:
    local_filename = 'cams-eac4_{}_sfc_2020.nc'.format(species)
    f = s3.download_file(bucket_name, 'data/air_pollution/cams/cams-eac4_{}_sfc_2020.nc'.format(species), local_filename)
    datasets[species] = xr.open_dataset(local_filename)

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir +'/data/boundaries/boundary_georef.csv')
boundary_georef

In [ ]:
def massfraction_to_concentration(massfraction):
    # input masses in kg, volumes in m^3
    # returns ug/m^3
    # 10^9 ug/kg
    # air density 1.223803 kg/m3 from https://confluence.ecmwf.int/display/UDOC/L60+model+level+definitions
    return massfraction * 1.223803 * 10**9

In [ ]:
def kilogrampersquaremeter_to_microgrampersquaremeter(conc):
    return conc * 10**9

In [ ]:
def exceedancedays(species, lon, lat):
    speciesdata = datasets[species]
    threshold = SPECIES_INFO[species]['who_threshold']
    localdata = speciesdata.sel(latitude=lat, longitude=lon, method='nearest')
    if SPECIES_INFO[species]['cams_unit'] == 'kg/kg':
        conc = massfraction_to_concentration(localdata)
    elif SPECIES_INFO[species]['cams_unit'] == 'kg/m^3':
        conc = kilogrampersquaremeter_to_microgrampersquaremeter(localdata)
    else:
        raise Exception('Unknown CAMS unit')
    dailymax = pd.DataFrame()
    dailymax['thedata'] = conc.to_array()[0]
    dailymax = dailymax.set_index(conc.time.to_index())
    dailymax = dailymax.resample('D').mean()
    exceeds = dailymax.thedata >= threshold
    return np.sum(exceeds), exceeds

In [ ]:
cams_multispecies_aq_indicator = pd.DataFrame()
for i in range(0, len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
    print("\n geo_name: "+boundary_id_aoi)
    
    # AOI
    boundary_id = boundary_id_aoi
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/data/boundaries/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo).select(['geo_parent_name','geo_name','geo_level','geo_id'])
    shape = shapely.geometry.shape(boundary_geo['features'][0]['geometry'])
    centroid = shape.centroid
    clon, clat = centroid.coords[0]  # Breaks if multipolygon
    df = geemap.ee_to_pandas(boundary_geo_ee)

    allspecies_exceeds = []
    for species in SPECIES_INFO:
        print(SPECIES_INFO[species]['name'])
        result, onespecies_exceeds = exceedancedays(species, clon, clat)
        allspecies_exceeds.append(onespecies_exceeds)
        df['exceedancedays_{}'.format(SPECIES_INFO[species]['name'])] = result
    df['exceedancedays_additive'] = pd.concat(allspecies_exceeds, axis=1).sum(axis=1).sum(axis=0)
    df['exceedancedays_atleastone'] = pd.concat(allspecies_exceeds, axis=1).sum(axis=1).gt(0).sum(axis=0)
    cams_multispecies_aq_indicator = pd.concat([cams_multispecies_aq_indicator, df])
    cams_multispecies_aq_indicator.to_csv(OUTPUT_FILENAME)

In [ ]:
pd.concat(allspecies_exceeds, axis=1).sum(axis=1).gt(0).sum(axis=0)

In [ ]:
processedcities = pd.read_csv(OUTPUT_FILENAME)
# ''out_dir +'/' OUTPUT_FILENAME'
processedcities

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
# aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'indicators/'+OUTPUT_FILENAME+''

processedcities.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')